<a href="https://colab.research.google.com/github/PRASANNA-416/Human-Activity-Detection-/blob/main/Human_Activity_Detection_MobileNet_v3_small_EXPERIMENTING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
import pandas as pd

# Load the training CSV file
train_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Training_set.csv'
train_data = pd.read_csv(train_csv_path)

# Load the testing CSV file
test_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/testing_set.csv'
test_data = pd.read_csv(test_csv_path)

In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CSV files
train_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Training_set.csv'
test_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/testing_set.csv'

train_data = pd.read_csv(train_csv_path)
test_data = pd.read_csv(test_csv_path)

# Define the target image size and batch size
target_size = (224, 224)  # Adjust according to your requirements
batch_size = 64

# Create ImageDataGenerators with preprocessing options
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values between 0 and 1
    # rotation_range=20,  # Randomly rotate images by 20 degrees
    # width_shift_range=0.2,  # Randomly shift the width by 20%
    # height_shift_range=0.2,  # Randomly shift the height by 20%
    # horizontal_flip=True  # Randomly flip images horizontally
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create train and test generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/train',  # Directory containing the train images
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/test',
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Check the class indices assigned by the generator
class_indices = test_generator.class_indices
print(class_indices)


Found 12100 validated image filenames belonging to 15 classes.
Found 500 validated image filenames belonging to 15 classes.
{'calling': 0, 'clapping': 1, 'cycling': 2, 'dancing': 3, 'drinking': 4, 'eating': 5, 'fighting': 6, 'hugging': 7, 'laughing': 8, 'listening_to_music': 9, 'running': 10, 'sitting': 11, 'sleeping': 12, 'texting': 13, 'using_laptop': 14}


In [12]:
import tensorflow as tf
import tensorflow_hub as hub

model = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5',
        trainable=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(15, activation='softmax')
])
#https://tfhub.dev/tensorflow/efficientnet/lite3/classification/2
model.build((None, 224, 224, 3)) 
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1024)              1026552   
                                                                 
 flatten_3 (Flatten)         (None, 1024)              0         
                                                                 
 dense_8 (Dense)             (None, 256)               262400    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 32)               

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Calculate the steps per epoch based on the desired batch size and the size of your dataset
steps_per_epoch = len(train_generator) // batch_size

# Update the number of epochs and steps_per_epoch
model.fit(train_generator, epochs=25, steps_per_epoch=190)


Epoch 1/25
190/190 [==============================] - 701s 4s/step - loss: 2.8141 - accuracy: 0.1405
Epoch 2/25
190/190 [==============================] - 672s 4s/step - loss: 2.4125 - accuracy: 0.2917
Epoch 3/25
190/190 [==============================] - 664s 3s/step - loss: 2.1179 - accuracy: 0.4068
Epoch 4/25
190/190 [==============================] - 664s 3s/step - loss: 1.9109 - accuracy: 0.4739
Epoch 5/25
190/190 [==============================] - 679s 4s/step - loss: 1.7500 - accuracy: 0.5277
Epoch 6/25
190/190 [==============================] - 679s 4s/step - loss: 1.6214 - accuracy: 0.5679
Epoch 7/25
190/190 [==============================] - 690s 4s/step - loss: 1.5187 - accuracy: 0.6063
Epoch 8/25
190/190 [==============================] - 683s 4s/step - loss: 1.4047 - accuracy: 0.6425
Epoch 9/25
190/190 [==============================] - 677s 4s/step - loss: 1.3001 - accuracy: 0.6729
Epoch 10/25
190/190 [==============================] - 676s 4s/step - loss: 1.1973 - accura

UnknownError: ignored

In [14]:




# Evaluate the model on the test generator
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

8/8 [==============================] - 7s 729ms/step - loss: 1.8680 - accuracy: 0.7180
Test Loss: 1.8680005073547363
Test Accuracy: 0.7179999947547913


In [15]:


model.save_weights('EfficientNetLite3.h5')

In [16]:

from google.colab import files
files.download('EfficientNetLite3.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>